In [1]:
import sys
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from pathlib import Path
from torchinfo import summary
from torchvision import transforms

from modules import data_setup, engine, utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

venv_dir = Path(sys.prefix)
project_root = venv_dir.parent
data_path = project_root/"data"

In [2]:
image_path = utils.download_data(
    source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
    destination="pizza_steak_sushi"
)

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    image_path/"train",
    image_path/"test",
    weights.transforms(),
    batch_size=32
)

In [3]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

for param in model.features.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True).to(device)
)
summary(model)

Layer (type:depth-idx)                                  Param #
EfficientNet                                            --
├─Sequential: 1-1                                       --
│    └─Conv2dNormActivation: 2-1                        --
│    │    └─Conv2d: 3-1                                 (864)
│    │    └─BatchNorm2d: 3-2                            (64)
│    │    └─SiLU: 3-3                                   --
│    └─Sequential: 2-2                                  --
│    │    └─MBConv: 3-4                                 (1,448)
│    └─Sequential: 2-3                                  --
│    │    └─MBConv: 3-5                                 (6,004)
│    │    └─MBConv: 3-6                                 (10,710)
│    └─Sequential: 2-4                                  --
│    │    └─MBConv: 3-7                                 (15,350)
│    │    └─MBConv: 3-8                                 (31,290)
│    └─Sequential: 2-5                                  --
│    │    └─MBConv

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

writer = utils.create_writer(
    experiment_name="data_10%",
    model_name="effnetb0",
    extra="5_epochs"
)

results = engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    epochs=5
)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.8383 | train_acc: 0.7969 | test_loss: 0.6988 | test_acc: 0.8968
Epoch: 2 | train_loss: 0.6831 | train_acc: 0.8516 | test_loss: 0.6696 | test_acc: 0.8551
Epoch: 3 | train_loss: 0.6941 | train_acc: 0.7617 | test_loss: 0.6412 | test_acc: 0.8040
Epoch: 4 | train_loss: 0.5864 | train_acc: 0.8008 | test_loss: 0.5810 | test_acc: 0.8258
Epoch: 5 | train_loss: 0.5475 | train_acc: 0.8242 | test_loss: 0.5528 | test_acc: 0.8258
